# Using ART on Traffic Sign Classifier

## Introduction:

ART is a library developed by "IBM Research" which is dedicated to adversarial machine learning. Its purpose is to allow rapid crafting and analysis of attacks and defense methods for machine learning models. ART provides an implementation for many state-of-the-art methods for attacking and defending classifiers. 

- ART Demo: https://art-demo.mybluemix.net
- ART Blog: https://www.ibm.com/blogs/research/2018/04/ai-adversarial-robustness-toolbox/
- ART Github: https://github.com/IBM/adversarial-robustness-toolbox


In this notebook, you will work with Adversarial Robustness Toolbox (ART) and try to implement an adversarial attack and its defense on a trained model.
You will work with a model trained on German Traffic Signs dataset and try to get the model to misclassify a stop sign.


## Dataset Citation

J. Stallkamp, M. Schlipsing, J. Salmen, and C. Igel. The German Traffic Sign Recognition Benchmark: A multi-class classification competition. In Proceedings of the IEEE International Joint Conference on Neural Networks, pages 1453–1460. 2011. <br>

@inproceedings{Stallkamp-IJCNN-2011,<br>
    author = {Johannes Stallkamp and Marc Schlipsing and Jan Salmen and Christian Igel},<br>
    booktitle = {IEEE International Joint Conference on Neural Networks},<br>
    title = {The {G}erman {T}raffic {S}ign {R}ecognition {B}enchmark: A multi-class classification competition},<br>
    year = {2011},<br>
    pages = {1453--1460}<br>
}

## Objective:

Upon completing the lab, you will learn:

- How to load a Tensorflow trained model
- How to create an ART classifier object using the loaded model
- How to perfom an adversarial attack
- How to perfom a defense to make sure manipulated images can still be classified correctly

## Library Versions:

- ART version 0.9.0
- Tensorflow version 1.15

## Instructions:

Please follow the steps in notebook below.

---
## Step 1: Load required libraries


In [ ]:
!pip install wget
import wget 

link_to_data = 'https://github.com/bleonardb3/Think2020/raw/master/Lab-3/Adversarial-Robustness-Toolbox-0.9.0.tar.gz'

!rm Adversarial-Robustness-Toolbox-0.9.0.tar.gz

wget.download(link_to_data)


In [ ]:
!pip install 'Adversarial-Robustness-Toolbox-0.9.0.tar.gz'

!pip install opencv-python
!pip install numpy

#### Here we load all the libraries needed to run this notebook.

In [ ]:
# common import for all cells
SEED=202

# standard libs
import pickle
import csv
from timeit import default_timer as timer
import os
from os.path import join, abspath, expanduser
import sys


#visualisation
%matplotlib inline

import matplotlib.pyplot as plt 
from IPython.display import Image
from IPython.display import display

# numerical Libraries 
import cv2
import math

import random
import numpy as np
random.seed(SEED)
np.random.seed(SEED)

import tensorflow as tf
tf.set_random_seed(SEED)
from tensorflow import keras
from tensorflow.python.training import moving_averages
from tensorflow.contrib.framework import add_model_variable
sess = tf.InteractiveSession()

import keras
import keras.backend as k
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing import image

# ART Libraries
from art.classifiers import TFClassifier
#from art.utils import load_dataset
#from art.attacks.fast_gradient import FastGradientMethod
from art.attacks import ProjectedGradientDescent
#from art.detection.detector import BinaryInputDetector

---
## Step 2: Load The Data

In this section we define a function to load our Traffic Sign dataset which is in "Pickle" file format.

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. These coordinates assume the original image. The pickled data, containes resized versions (32 x 32) of these images.

## INSERT OBJECT STORAGE CREDENTIALS
----


#### a. Please insert the IBM Cloud Object Storage credentials in the cell below.

In [ ]:
credentials={
#   'apikey': "Bk7zOKRYr9qVkZyFmZQUGDH0bZ-a-PBargE1nGKqedOT",
#   'endpoints': "https://s3-api.us-geo.objectstorage.service.networklayer.com",
#   'resource_instance_id': "crn:v1:bluemix:public:cloud-object-storage:global:a/745879b6e1314c84b790bf90e21994b3:49656519-f026-4560-8b46-a29457efa1dc::",
#   'BUCKET': 'watsonstudiolabs-donotdelete-pr-ptznjdche7dyeb',
   'apikey': "XsWGp-u7xhQmdcJFQm2-EPhWPNU1boOlHo8VP0n8W6uT",
   'endpoints': "https://s3-api.us-geo.objectstorage.service.networklayer.com",
   'resource_instance_id': "crn:v1:bluemix:public:cloud-object-storage:global:a/6111bdc91ee705970b7d8b4f0188cbfd:838529f7-eaf2-4f3b-9ec1-d5ac228a1bcd::",
   'BUCKET': 'adversarialrobustnesstoolkit-donotdelete-pr-gfodsg09fvjys0',
   'FILE_TRAIN': 'train.p',
   'FILE_VALID': 'valid.p',
   'FILE_TEST': 'test.p',
   'FILE_SIGN': 'signnames.csv',
   'FILE_SIGN_ALL': 'signnames_all.jpg',
   'MODEL_DATA': 'final.ckpt.data-00000-of-00001',
   'MODEL_CHECKPOINT': 'checkpoint',
   'MODEL_CKPT_INDEX': 'final.ckpt.index',
   'MODEL_META': 'final.ckpt.meta',
   'FILE_STOP': 'stop_001.png',
   'FILE_DIAG_1': '001.png',
   'FILE_DIAG_2': '002.png'
}


#### b. Following cell contains code for initiating a connection to the cloud object storage.

In [ ]:
from ibm_botocore.client import Config
import ibm_boto3
cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['apikey'],
    ibm_service_instance_id=credentials['resource_instance_id'],
    ibm_auth_endpoint='https://iam.ng.bluemix.net/oidc/token',
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['endpoints'])

#### c. Following cell contains code to download the dataset and image files required for the lab.

In [ ]:

!mkdir checkpoints
!mkdir checkpoints/final.ckpt
res=cos.download_file(Bucket=credentials['BUCKET'],Key=credentials['FILE_TRAIN'],Filename='training_file')
res=cos.download_file(Bucket=credentials['BUCKET'],Key=credentials['FILE_VALID'],Filename='valid_file')
res=cos.download_file(Bucket=credentials['BUCKET'],Key=credentials['FILE_TEST'],Filename='testing_file')
res=cos.download_file(Bucket=credentials['BUCKET'],Key=credentials['FILE_SIGN'],Filename='classname_file')
res=cos.download_file(Bucket=credentials['BUCKET'],Key=credentials['FILE_SIGN_ALL'],Filename='signnames_all_file')
res=cos.download_file(Bucket=credentials['BUCKET'],Key=credentials['MODEL_META'],Filename='checkpoints/final.ckpt.meta')
res=cos.download_file(Bucket=credentials['BUCKET'],Key=credentials['MODEL_DATA'],Filename='checkpoints/final.ckpt.data-00000-of-00001')
res=cos.download_file(Bucket=credentials['BUCKET'],Key=credentials['MODEL_CHECKPOINT'],Filename='checkpoints/checkpoint')
res=cos.download_file(Bucket=credentials['BUCKET'],Key=credentials['MODEL_CKPT_INDEX'],Filename='checkpoints/final.ckpt.index')
res=cos.download_file(Bucket=credentials['BUCKET'],Key=credentials['FILE_STOP'],Filename='stop_001.png')
res=cos.download_file(Bucket=credentials['BUCKET'],Key=credentials['FILE_DIAG_1'],Filename='diagram1.png')
res=cos.download_file(Bucket=credentials['BUCKET'],Key=credentials['FILE_DIAG_2'],Filename='diagram2.png')


#### d. Following cell contains code for loading the trainset and test set, and then Splitting them in x_train, y_train, x_test and y_test.

In [ ]:
# Load pickled data  

def load_data(): 
    training_file  = 'training_file'
    testing_file   = 'testing_file'
    classname_file = 'classname_file'

    classnames = []
    with open(classname_file) as _f:
        rows = csv.reader(_f, delimiter=',')
        next(rows, None)  # skip the headers
        for i, row in enumerate(rows):
            assert(i==int(row[0]))
            classnames.append(row[1])
 
    with open(training_file, mode='rb') as f:
        train = pickle.load(f)
    with open(testing_file, mode='rb') as f:
        test = pickle.load(f)

    X_train, y_train = train['features'], train['labels']
    X_test, y_test   = test['features'], test['labels']
    
    
    X_train  = X_train.astype(np.float32)
    y_train  = y_train.astype(np.int32)
    X_test   = X_test.astype(np.float32)
    y_test   = y_test.astype(np.int32)
    
    return  classnames, X_train, y_train, X_test, y_test 


---

## Step 3: Dataset Summary & Exploration

In the visualization section you will see representation of each class as well as number of samples in each class.


In [ ]:
classnames, X_train, y_train, X_test, y_test = load_data() 
 
# Number of training examples 
num_train = len(X_train)

# Number of testing examples.
num_test = len(X_test)

# the shape of an traffic sign image
_, height, width, channel = X_train.shape
image_shape = (height, width, channel)

# Number of unique classes/labels in the dataset
num_class = len(np.unique(y_train))


print("Number of training examples =", num_train )
print("Number of testing examples =", num_test )
print("Image data shape =", image_shape)
print("Number of classes =", num_class)

### Visualization

In this section, you will visualize the dataset to see what different classes look like.
Also, you demonstrate the sample count of each class.

This Section might take couple of seconds to minutes to complete (based on how powerful the system you are working on is)

In [ ]:
# Some functions to help us draw graphs

def get_label_image(c): 
    img=cv2.imread('signnames_all_file',1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    H, W, _ = img.shape
    dH = H/7.
    dW = W/7.105
    y = c//7
    x = c%7 
    label_image = img[round(y*dH):round(y*dH+dH), round(x*dW):round(x*dW+dW),:]
    label_image = cv2.resize(label_image, (0,0), fx=32./dW, fy=32./dH,)
    return label_image


def insert_subimage(image, sub_image, y, x): 
    h, w, c = sub_image.shape
    image[y:y+h, x:x+w, :]=sub_image 
    return image


# Start Visualizing the dataset

train_images, train_labels = X_train, y_train

#results image
num_sample=10
results_image = 255.*np.ones(shape=(num_class*height,(num_sample+2+22)*width, channel),dtype=np.float32)
for c in range(num_class):
    label_image = get_label_image(c)
    insert_subimage(results_image, label_image, c*height, 0)

    #make mean
    idx = list(np.where(train_labels== c)[0])
    mean_image = np.average(train_images[idx], axis=0)
    insert_subimage(results_image, mean_image, c*height, width)

    #make random sample
    for n in range(num_sample):
        sample_image = train_images[np.random.choice(idx)]
        insert_subimage(results_image, sample_image, c*height, (2+n)*width)

    #print summary
    count=len(idx)
    percentage = float(count)/float(len(train_images))
    cv2.putText(results_image, '%02d:%-6s'%(c, classnames[c]), ((2+num_sample)*width, int((c+0.7)*height)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1)
    cv2.putText(results_image, '[%4d]'%(count), ((2+num_sample+14)*width, int((c+0.7)*height)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1)
    cv2.rectangle(results_image,((2+num_sample+16)*width, c*height),((2+num_sample+16)*width + round(percentage*3000), (c+1)*height),(0,0,255),-1)


print('** training data summary **')
print('\t1st column: label(image)')
print('\t2nd column: mean image')
print('\tother column: example images')
print('\tblack text: label')
print('\tblue text: sample count for each class and histogram plot')
plt.rcParams["figure.figsize"] = (25,25)
plt.imshow(results_image.astype(np.uint8))
plt.axis('off') 
plt.show()

----

## Step 4: Defining Model Architecture



In this section we define our DNN layers and architecture.
This model is a modified Densenet architecture. [3]


In [ ]:
##  global varaiables ##
IS_TRAIN_PHASE = tf.placeholder(dtype=tf.bool, name='is_train_phase')

def conv2d(input, num_kernels=1, kernel_size=(1,1), stride=[1,1,1,1], padding='SAME', has_bias=True, name='conv'):

    input_shape = input.get_shape().as_list()
    assert len(input_shape)==4
    C = input_shape[3]
    H = kernel_size[0]
    W = kernel_size[1]
    K = num_kernels

    ##[filter_height, filter_width, in_channels, out_channels]
    w    = tf.get_variable(name=name+'_weight', shape=[H, W, C, K], initializer=tf.truncated_normal_initializer(stddev=0.1))
    conv = tf.nn.conv2d(input, w, strides=stride, padding=padding, name=name)
    if has_bias:
        b = tf.get_variable(name=name + '_bias', shape=[K], initializer=tf.constant_initializer(0.0))
        conv = conv+b

    return conv


def relu(input, name='relu'):
    act = tf.nn.relu(input, name=name)
    return act

def prelu(input, name='prelu'):
    alpha = tf.get_variable(name=name+'_alpha', shape=input.get_shape()[-1],
                       #initializer=tf.constant_initializer(0.25),
                        initializer=tf.random_uniform_initializer(minval=0.1, maxval=0.3),
                        dtype=tf.float32)
    pos = tf.nn.relu(input)
    neg = alpha * (input - abs(input)) * 0.5

    return pos + neg


# very leaky relu
def vlrelu(input, alpha=0.25, name='vlrelu'): #  alpha between 0.1 to 0.5
    act =tf.maximum(alpha*input,input)
    return act

def maxpool(input, kernel_size=(1,1), stride=[1,1,1,1], padding='SAME', has_bias=True, name='max' ):
    H = kernel_size[0]
    W = kernel_size[1]
    pool = tf.nn.max_pool(input, ksize=[1, H, W, 1], strides=stride, padding=padding, name=name)
    return pool

def avgpool(input, kernel_size=(1,1), stride=[1,1,1,1], padding='SAME', has_bias=True, is_global_pool=False, name='avg'):

    if is_global_pool==True:
        input_shape = input.get_shape().as_list()
        assert len(input_shape) == 4
        H = input_shape[1]
        W = input_shape[2]

        pool = tf.nn.avg_pool(input, ksize=[1, H, W, 1], strides=[1,H,W,1], padding='VALID', name=name)
        pool = flatten(pool)

    else:
        H = kernel_size[0]
        W = kernel_size[1]
        pool = tf.nn.avg_pool(input, ksize=[1, H, W, 1], strides=stride, padding=padding, name=name)

    return pool


def dropout(input, keep=1.0, name='drop'):
    #drop = tf.cond(IS_TRAIN_PHASE, lambda: tf.nn.dropout(input, keep), lambda: input)
    drop = tf.cond(IS_TRAIN_PHASE,
                   lambda: tf.nn.dropout(input, keep),
                   lambda: tf.nn.dropout(input, 1))
    return drop


def flatten(input, name='flat'):
    input_shape = input.get_shape().as_list()        # list: [None, 9, 2]
    dim   = np.prod(input_shape[1:])                 # dim = prod(9,2) = 18
    flat  = tf.reshape(input, [-1, dim], name=name)  # -1 means "all"
    return flat

def concat(input, name='cat'):
    cat = tf.concat(axis=3, values=input, name=name)
    return cat


#https://github.com/tensorflow/tensorflow/blob/master/tensorflow/g3doc/api_docs/python/functions_and_classes/shard4/tf.contrib.layers.batch_norm.md
#http://www.bubufx.com/detail-1792794.html
def bn (input, decay=0.9, eps=1e-5, name='bn'):
    with tf.variable_scope(name) as scope:
        bn = tf.cond(IS_TRAIN_PHASE,
            lambda: tf.contrib.layers.batch_norm(input,  decay=decay, epsilon=eps, center=True, scale=True,
                              is_training=1,reuse=None,
                              updates_collections=None, scope=scope),
            lambda: tf.contrib.layers.batch_norm(input, decay=decay, epsilon=eps, center=True, scale=True,
                              is_training=0, reuse=True,
                              updates_collections=None, scope=scope))

    return bn


# basic building blocks

def conv2d_bn_relu(input, num_kernels=1, kernel_size=(1,1), stride=[1,1,1,1], padding='SAME', name='conv'):
    with tf.variable_scope(name) as scope:
        block = conv2d(input, num_kernels=num_kernels, kernel_size=kernel_size, stride=stride, padding=padding, has_bias=False)
        block = bn(block)
        block = relu(block)
    return block


def bn_relu_conv2d (input, num_kernels=1, kernel_size=(1, 1), stride=[1, 1, 1, 1], padding='SAME', name='conv'):
    with tf.variable_scope(name) as scope:
        block = bn(input)
        block = relu(block)
        block = conv2d(block, num_kernels=num_kernels, kernel_size=kernel_size, stride=stride, padding=padding, has_bias=False)
    return block



# modified dense block from the paper [1] "Densely Connected Convolutional Networks" - Gao Huang, Zhuang Liu, Kilian Q. Weinberger, 
# Laurens van der Maaten, Arxiv 2016
# Modification: 
#   1. the paper uses bn-relu-conv but we use conv-bn-relu
#   2. the paper uses dropout inside the block but we shift the dropout outside the block see network construction later
def dense_block_cbr (input, num=1, num_kernels=1, kernel_size=(1, 1), drop=None, name='DENSE'):
 
    block = input
    for n in  range(num):
        with tf.variable_scope(name+'_%d'%n) as scope:
            conv = conv2d(block, num_kernels=num_kernels, kernel_size=kernel_size, stride=[1,1,1,1], padding='SAME', has_bias=False)
            conv = bn(conv)
            conv = relu(conv)

            if drop is not None:
                keep = (1 - drop) ** (1. / num)
                conv = dropout(conv, keep=keep)

            block = concat((block, conv))
    return block

# the loss 
def l2_regulariser(decay):

    variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    for v in variables:
        name = v.name
        if 'weight' in name:  #this is weight
            l2 = decay * tf.nn.l2_loss(v)
            tf.add_to_collection('losses', l2)
        elif 'bias' in name:  #this is bias
            pass
        elif 'beta' in name:
            pass
        elif 'gamma' in name:
            pass
        elif 'moving_mean' in name:
            pass
        elif 'moving_variance' in name:
            pass
        elif 'moments' in name:
            pass

        else:
            #pass
            #raise Exception('unknown variable type: %s ?'%name)
            pass

    l2_loss = tf.add_n(tf.get_collection('losses'))
    return l2_loss


def cross_entropy(logit, label, name='cross_entropy'):
    label = tf.cast(label, tf.int64)
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=label), name=name)
    return cross_entropy


def accuracy(prob, label, name='accuracy'):
    correct_prediction = tf.equal(tf.argmax(prob, 1), tf.cast(label, tf.int64))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name=name)
    return accuracy



In [ ]:
# my densenet here!
# the inference part (without loss)

def DenseNet_3( input_shape=(1,1,1), output_shape = (1)):

    H, W, C   = input_shape
    num_class = output_shape
    input     = tf.placeholder(shape=[None, H, W, C], dtype=tf.float32, name='input')

    #color preprocessing using conv net:
    #see "Systematic evaluation of CNN advances on the ImageNet"-Dmytro Mishkin, Nikolay Sergievskiy, Jiri Matas, ARXIV 2016
    # https://arxiv.org/abs/1606.02228
    # we use learnable prelu (different from paper) and 3x3 onv
    with tf.variable_scope('preprocess') as scope:
        input = bn(input, name='b1')
        input = conv2d(input, num_kernels=8, kernel_size=(3, 3), stride=[1, 1, 1, 1], padding='SAME', has_bias=True, name='c1')
        input = prelu(input, name='r1')
        input = conv2d(input, num_kernels=8, kernel_size=(1, 1), stride=[1, 1, 1, 1], padding='SAME', has_bias=True, name='c2')
        input = prelu(input, name='r2')


    with tf.variable_scope('block1') as scope:
        block1 = conv2d_bn_relu(input, num_kernels=32, kernel_size=(5, 5), stride=[1, 1, 1, 1], padding='SAME')
        block1 = maxpool(block1, kernel_size=(2,2), stride=[1, 2, 2, 1], padding='SAME')

    # we use conv-bn-relu in DENSE block (different from paper)
    # dropout is taken out of the block
    with tf.variable_scope('block2') as scope:
        block2 = dense_block_cbr(block1, num=4, num_kernels=16, kernel_size=(3, 3), drop=None)
        block2 = maxpool(block2, kernel_size=(2, 2), stride=[1, 2, 2, 1], padding='SAME')

    with tf.variable_scope('block3') as scope:
        block3 = dense_block_cbr(block2, num=4, num_kernels=24, kernel_size=(3, 3), drop=None)
        block3 = dropout(block3, keep=0.9)
        block3 = maxpool(block3,  kernel_size=(2,2), stride=[1, 2, 2, 1], padding='SAME')

    with tf.variable_scope('block4') as scope:
        block4 = dense_block_cbr(block3, num=4, num_kernels=32, kernel_size=(3, 3), drop=None)
        block4 = conv2d_bn_relu(block4, num_kernels=num_class, kernel_size=(1,1), stride=[1, 1, 1, 1], padding='SAME')
        block4 = dropout(block4, keep=0.8)
        block4 = avgpool(block4, is_global_pool=True)


    logit = block4
    return logit


### An ilustration on what the network acually looks like (Visually)

In [ ]:
img1=Image(filename="diagram1.png")
img2=Image(filename="diagram2.png")
display(img1, img2)

---
## Step 5: Using ART to Attack & Defense

In this section you will load the trained model and start working with ART. 

- You will first define a TFClassifier which is an ART object that will be used as the model when you are working with ART.
- After defining the TFClassifier, you will try it on a clean (non-adversarial) image and see the prediction. 
- Then you will craft an Adversarial image using "Projected Gradient Descent method" and feed it to the model to see its prediction.
- Finally, you will use a defense method called "Feature Squeezing Defence" on the adversarial image to neutralize it. Then you will feed it to the model and see the result.

### 5.1. Defining model parameters and loading the trained model

In [ ]:
adv_make_net = DenseNet_3

adv_logit  = adv_make_net(input_shape =(32, 32, 3), output_shape=(num_class))
adv_data   = tf.get_default_graph().get_tensor_by_name('input:0')
adv_label  = tf.placeholder(dtype=tf.int32, shape=[None, 43])
adv_prob   = tf.nn.softmax(adv_logit)

adv_l2     = l2_regulariser(decay=0.0005)
adv_loss   = cross_entropy(adv_logit, adv_label)
adv_metric = accuracy(adv_prob, adv_label)

adv_learning_rate = tf.placeholder(tf.float32, shape=[])
adv_solver = tf.train.MomentumOptimizer(learning_rate=adv_learning_rate, momentum=0.9)
adv_solver_step = adv_solver.minimize(adv_loss)

adv_sess = tf.Session()

adv_saver  = tf.train.Saver()
adv_saver.restore(adv_sess, 'checkpoints/final.ckpt')

### 5.2. Defining TFClassifier

TFClassifier is an ART object that containes the trained model.

In [ ]:
# classifier = TFClassifier(input_ph=adv_data, output = adv_logit, labels_ph=adv_label, train=adv_solver_step, loss=adv_loss, learning = IS_TRAIN_PHASE, sess=adv_sess, preprocessing= (0, 1))
classifier = TFClassifier(adv_data, adv_logit, output_ph=adv_label, train=adv_solver_step, loss=adv_loss, learning = IS_TRAIN_PHASE, sess=adv_sess, preprocessing= (0, 1))

classifier.set_learning_phase(train=False)

In [ ]:
print(classifier)

### 5.3. Loading a "Clean" test image and test the TFClassifier on it

You can see that it detected that by 100% confidence it is a stop sign --> Correctly classified the image.

In [ ]:
from keras.preprocessing import image

init_img = np.zeros(shape=(1,32,32,3),dtype=np.float32)

image_file = join('stop_001.png')
image_ = image.load_img(image_file, target_size=(32, 32))
init_img = image.img_to_array(image_)
img = init_img[None , ...]

pred = classifier.predict(img)
label = np.argmax(pred, axis=1)[0]
confidence = pred[:,label][0]
print('Prediction:', classnames[label], '- confidence {0:.0f}%'.format(100*confidence))
plt.figure(figsize=(5,5))
plt.imshow(init_img / 255)



# 5.4. Perform Attack

Here we will use "Projected Gradient Descent" attack. The PGD attack is a white-box attack (meaning the attacker has access to the model gradients i.e. the attacker has a copy of your model’s weights). This threat model gives the attacker much more power than black box attacks as they can specifically craft their attack to fool your model without having to rely on transfer attacks that often result in human-visible perturbations. PGD can be considered the most “complete” white-box adversary as it lifts any constraints on the amount of time and effort the attacker can put into finding the best attack.

you will perform following steps in this section:

1- Generate an adversarial image using "Projected Gradient Descent"
   - First generate a carefully crafted noise
   - Then apply it to the original image to craft the adversarial image
   
2- Feed the Adversary to the classifier and evaluate it on the trained model

3- Finally we visualize the images (original, the crafted noise, adversarial image and a representation of the class that was predicted based on the adversarial image)

Reference Link to research paper on the attack: https://arxiv.org/abs/1706.06083


In [ ]:
adv = ProjectedGradientDescent(classifier, targeted=False, eps_step=1, eps=2, max_iter=50)

# Generate attack image
img_adv = adv.generate(img)

# Evaluate it on model
pred_adv = classifier.predict(img_adv)
label_adv = np.argmax(pred_adv, axis=1)[0]
confidence_adv = pred_adv[:, label_adv][0]
print('Prediction:', classnames[label_adv], '- confidence {0:.0f}%'.format(100*confidence_adv) ) 

fig = plt.figure(figsize=(16,16))

a = fig.add_subplot(1, 4, 1)
a.set_title('Original Predicted Sign\nwith {0:.0f}% Accuracy'.format(100*confidence), fontsize=12)
imgplot = plt.imshow(img[0] / 255)
imgplot.set_clim(0.0, 0.7)

a = fig.add_subplot(1, 4, 2)
a.set_title('+      Noise      ---->', fontsize=12)
imgplot = plt.imshow(np.abs(img[0] - img_adv[0]) )

a = fig.add_subplot(1, 4, 3)
a.set_title('Adversarial Image  ---->', fontsize=12)
imgplot = plt.imshow(img_adv[0] / 255)

a = fig.add_subplot(1, 4, 4)
a.set_title('Predicted Sign\n with {0:.0f}% Accuracy'.format(100*confidence_adv), fontsize=12)
im = get_label_image(int(label_adv))
imL = cv2.resize(im, dsize=(32, 32), interpolation=cv2.INTER_LINEAR)
imgL = np.float32(imL)
imgplot = plt.imshow(imgL / 255)



### 5.5. Apply Defense

In this section You will apply a defense against the adversarial image. Here we assume that we already know the input is manipulated and we want to neutralize it in a way that the DNN can correctly classify it as "Stop Sign".

You will use "Feature Squeezing" defense to neutralize the adversarial image. Feature squeezing reduces the search space available to an adversary by coalescing samples that correspond to many different feature vectors in the original space into a single sample. By comparing a DNN model’s prediction on the original input with that on squeezed inputs, feature squeezing detects adversarial examples with high accuracy.

This method is based on paper named "Feature Squeezing: Detecting Adversarial Examples in Deep Neural Networks" by Weilin Xu, David Evans, Yanjun Qi from University of Virginia. Link :   https://arxiv.org/abs/1704.01155

In [ ]:
from art.defences import FeatureSqueezing

fs = FeatureSqueezing(bit_depth=3, clip_values=(0, 255))
img_def = fs(img_adv)
pred_def = classifier.predict(img_def[0])
label_def = np.argmax(pred_def, axis=1)[0]
confidence_def = pred_def[:, label_def][0]
print('Prediction:', classnames[label_def], '- confidence {0:.2f}%'.format(100*confidence_def))

fig = plt.figure(figsize=(15,15))

a = fig.add_subplot(1, 3, 1)
imgplot = plt.imshow(img[0] / 255)
imgplot.set_clim(0.0, 0.7)
a.set_title('Original Image', fontsize = 12)

a = fig.add_subplot(1, 3, 2)
imgplot = plt.imshow(img_adv[0] / 255)
a.set_title('Adversarial Image\n  Misclassified as "Ahead Only"', fontsize=12)

a = fig.add_subplot(1, 3, 3)
imgplot = plt.imshow(img_def[0][0] / 255)
imgplot.set_clim(0.0, 0.7)
a.set_title('Cleaned Image\n Correctly classified with {0:.0f}% Accuracy'.format(100*confidence_def), fontsize=12)

### References

[1] The Traffic sign classifier uses German Traffic Sign dataset: http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset

[2] The code to define and train the model was used from the code in https://github.com/hengck23-udacity/udacity-driverless-car-nd-p2

[3] "Densely Connected Convolutional Networks" - Gao Huang, Zhuang Liu, Kilian Q. Weinberger, Laurens van der Maaten, Arxiv 2016